# Imports

In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta

# Connection Data

In [2]:
schema="e_scooter"   # name of the database you want to use here
host="127.0.0.1"        # to connect to your local server
user="root"
password="YOUR_PASSWORD_HERE" # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# Cities

Making a cities DataFrame from a dictionary we created

In [3]:
cities_dict = {
  'city': ['Berlin', 'Munich', 'Cologne', 'Leipzig'],
  'city_ascii': ['Berlin', 'Munich', 'Cologne', 'Leipzig'],
  'lat': [52.5167, 48.1375, 50.9422, 51.3333],
  'lng': [13.3833, 11.575, 6.9578, 12.3833],
  'country': ['Germany', 'Germany', 'Germany', 'Germany'],
  'iso2': ['DE', 'DE', 'DE', 'DE'],
  'iso3': ['DEU', 'DEU', 'DEU', 'DEU'],
  'admin_name': ['Berlin', 'Bavaria', 'North Rhine-Westphalia', 'Saxony'],
  'population': [3664088, 1488202, 1083498, 597215]
 }

cities_df = pd.DataFrame(cities_dict)

In [4]:
cities_df.to_sql("cities", if_exists="append", con=con, index=False)

4

# Weather

In [5]:
city_list = ["Berlin", "Munich", "Leipzig", "Cologne"]

In [6]:
def weather(c_list):

  df_list = []

  for city in c_list:

    API_key = "YOUR_API_KEY_HERE"

    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric"

    response = requests.get(url)

    json = response.json()

    for item in json["list"]:
      weather_dict = {
        "city_name": [],
        "country": [],
        "rain": [],
        "temperature": [],
        "time_for_forecast": []
    }
      try:
        weather_dict["rain"].append(item["rain"]["3h"])
      except:
        weather_dict["rain"].append("0")
      weather_dict["city_name"].append(city)
      weather_dict["country"].append(json["city"]["country"])
      weather_dict["temperature"].append(item["main"]["temp"])
      weather_dict["time_for_forecast"].append(item['dt_txt'])

      weather_df = pd.DataFrame(weather_dict)

      weather_df["time_for_forecast"] = pd.to_datetime(weather_df["time_for_forecast"])

      weather_df.loc[weather_df["city_name"] == "Berlin", "city_name"] = 1
      weather_df.loc[weather_df["city_name"] == "Leipzig", "city_name"] = 4
      weather_df.loc[weather_df["city_name"] == "Cologne", "city_name"] = 3
      weather_df.loc[weather_df["city_name"] == "Munich", "city_name"] = 2

      weather_df.rename(columns={"city_name": "cityid"}, inplace=True)

      df_list.append(weather_df)

  return pd.concat(df_list)

In [7]:
weather_df = weather(city_list)

In [8]:
weather_df.to_sql("weather", if_exists="append", con=con, index=False)

160

# Airports

In [9]:
airports_dict = {
    "cityid": [1, 2, 3, 4],
    "icao": ["EDDB", "EDDM", "EDDK", "EDDP"],
}

airports_df = pd.DataFrame(airports_dict)

In [10]:
airports_df.to_sql("airports", if_exists="append", con=con, index=False)

4

# Flights

In [11]:
icao_codes = ["EDDB", "EDDM", "EDDP", "EDDK"]

In [12]:
def arrivals(codes):

  arrivals_list = []

  today = datetime.now().date()
  tomorrow = today + timedelta(days=1)
  
  morning = {"start_time": "00:00", "end_time": "12:00"}
  afternoon = {"start_time": "12:00", "end_time": "23:59"}
  times = [morning, afternoon]

  for code in codes:
    for time in times:

      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{code}/{tomorrow}T{time['start_time']}/{tomorrow}T{time['end_time']}"

      querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

      headers = {
        "X-RapidAPI-Key": "YOUR_API_KEY_HERE",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
      }

      response = requests.request("GET", url, headers=headers, params=querystring)
      arrivals_df = pd.json_normalize(response.json()["arrivals"]).copy()

      arrivals_new_df = arrivals_df.loc[:, ['arrival.scheduledTimeLocal', 'number', "status", 'aircraft.model', 'airline.name', 'codeshareStatus']]

      arrivals_new_df["arrival.scheduledTimeLocal"] = pd.to_datetime(arrivals_new_df["arrival.scheduledTimeLocal"])

      #arrivals_new_df["arrival_icao"] = code # alternative way to add the airport code

      arrivals_new_df.insert(0, "arrival_icao", code)

      arrivals_list.append(arrivals_new_df)

  return pd.concat(arrivals_list)

In [13]:
flights_df = arrivals(icao_codes)

In [14]:
flights_df.to_sql("flights", if_exists="append", con=con, index=False)

617